# Module 08: Advanced Oscillators - Stochastic & Williams %R

---

## Welcome Back! 

In Module 07, you mastered volatility analysis with Bollinger Bands and ATR. Now we're diving into **oscillators** - indicators that help identify overbought/oversold conditions and momentum shifts.

### Learning Objectives

By the end of this module, you will be able to:
- Understand how oscillators measure momentum
- Calculate and interpret the Stochastic Oscillator
- Use Williams %R for momentum analysis
- Detect divergences (hidden trading opportunities)
- Combine multiple oscillators for stronger signals
- Apply oscillator-based trading strategies

### Prerequisites

- Completed Modules 00-07
- Understanding of overbought/oversold concepts
- Familiarity with RSI and MACD (from Module 04)

### Time Required

60-75 minutes

---

## Part 1: Understanding Oscillators

### What are Oscillators?

**Oscillators** are technical indicators that fluctuate between fixed bounds (usually 0-100). They help identify:
- Overbought conditions (potential sell signals)
- Oversold conditions (potential buy signals)
- Momentum shifts
- Divergences from price

### Real-World Analogy

Think of oscillators like a **pendulum**:
- When it swings too far right (overbought) → It must swing back left
- When it swings too far left (oversold) → It must swing back right
- The center represents equilibrium
- The speed of swing shows momentum

### Oscillators vs Trend Indicators

| Feature | Oscillators | Trend Indicators |
|---------|-------------|------------------|
| **Range** | Bounded (0-100) | Unbounded |
| **Best For** | Ranging markets | Trending markets |
| **Signals** | Overbought/oversold | Trend direction |
| **Examples** | RSI, Stochastic, Williams %R | Moving Averages, MACD |

### Why Multiple Oscillators?

Different oscillators have different strengths:
- **RSI** = Fast, responsive (from Module 04)
- **Stochastic** = Shows momentum + direction
- **Williams %R** = Very sensitive, catches early reversals

**Key Principle**: When multiple oscillators agree, the signal is stronger!

---

## Part 2: Setting Up Our Environment

Let's import libraries and fetch data for UUE Holdings.

In [ ]:
# Import required libraries
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas_ta as ta
import warnings

warnings.filterwarnings("ignore")

# Set up plotting style
%matplotlib inline
try:
    plt.style.use("seaborn-v0_8-darkgrid")
except:
    try:
        plt.style.use("seaborn-darkgrid")
    except:
        plt.style.use("default")

sns.set_palette("husl")

print("All libraries imported successfully!")
print(f"Ready to analyze oscillators!")

In [ ]:
# Fetch 1 year of data for UUE Holdings Berhad\nticker = "0310.KL"  # UUE Holdings\nprint(f"Fetching data for {ticker}...\\n")\n\ntry:\n    data = yf.download(ticker, period="1y", progress=False)\n    \n    # Flatten column names if MultiIndex\n    if isinstance(data.columns, pd.MultiIndex):\n        data.columns = data.columns.get_level_values(0)\n    \n    if data.empty:\n        print("No data received.")\n    else:\n        print(f"Successfully fetched {len(data)} days of data!")\n        print(f"Date range: {data.index[0].date()} to {data.index[-1].date()}")\n        print(f"Latest close: RM {data['Close'].iloc[-1]:.2f}\\n")\n        print(data.head())\nexcept Exception as e:\n    print(f"Error: {e}")

---\n\n## Part 3: The Stochastic Oscillator\n\n### What is the Stochastic Oscillator?\n\nCreated by George Lane in the 1950s, the **Stochastic Oscillator** compares a stock's closing price to its price range over a period.\n\n**Formula**:\n- **%K** = (Current Close - Lowest Low) / (Highest High - Lowest Low) × 100\n- **%D** = 3-day SMA of %K (signal line)\n\n**Default Settings**: 14, 3, 3\n- 14 = Look-back period\n- 3 = %K smoothing\n- 3 = %D smoothing\n\n### Interpretation\n\n**Range**: 0 to 100\n- **Above 80** = Overbought (potential sell)\n- **Below 20** = Oversold (potential buy)\n- **%K crosses above %D** = Bullish signal\n- **%K crosses below %D** = Bearish signal\n\n### Real-World Analogy\n\nThink of Stochastic like **room temperature**:\n- Value of 20 = Very cold (stock oversold, may heat up)\n- Value of 80 = Very hot (stock overbought, may cool down)\n- Value of 50 = Comfortable (neutral zone)\n\n---

In [ ]:
# Calculate Stochastic Oscillator manually\nif 'data' in locals() and not data.empty:\n    period = 14\n    smooth_k = 3\n    smooth_d = 3\n    \n    # Calculate %K (fast stochastic)\n    lowest_low = data['Low'].rolling(window=period).min()\n    highest_high = data['High'].rolling(window=period).max()\n    \n    data['Stoch_K'] = ((data['Close'] - lowest_low) / (highest_high - lowest_low)) * 100\n    \n    # Smooth %K to get slow stochastic\n    data['Stoch_K'] = data['Stoch_K'].rolling(window=smooth_k).mean()\n    \n    # Calculate %D (signal line)\n    data['Stoch_D'] = data['Stoch_K'].rolling(window=smooth_d).mean()\n    \n    print("Stochastic Oscillator calculated successfully!\\n")\n    print("Last 5 days:")\n    print(data[['Close', 'Stoch_K', 'Stoch_D']].tail())\n    \n    # Current reading\n    latest_k = data['Stoch_K'].iloc[-1]\n    latest_d = data['Stoch_D'].iloc[-1]\n    \n    print(f"\\nCurrent Stochastic:")\n    print(f"%K: {latest_k:.2f}")\n    print(f"%D: {latest_d:.2f}")\n    \n    if latest_k > 80:\n        print("\\nOVERBOUGHT condition - Consider selling")\n    elif latest_k < 20:\n        print("\\nOVERSOLD condition - Consider buying")\n    else:\n        print("\\nNEUTRAL condition")\nelse:\n    print("Please run data fetching cell first!")

---\n\n## Part 4: Williams %R\n\n### What is Williams %R?\n\nDeveloped by Larry Williams, **Williams %R** measures overbought/oversold levels. It's similar to Stochastic but:\n- Range: 0 to -100 (inverted scale)\n- More sensitive (faster signals)\n- Single line (no %D)\n\n**Formula**:\nWilliams %R = [(Highest High - Current Close) / (Highest High - Lowest Low)] × -100\n\n**Default Period**: 14 days\n\n### Interpretation\n\n**Range**: 0 to -100\n- **Above -20** = Overbought (potential sell)\n- **Below -80** = Oversold (potential buy)\n- **-50** = Neutral zone\n\n### Williams %R vs Stochastic\n\n| Feature | Williams %R | Stochastic |\n|---------|-------------|------------|\n| **Lines** | 1 line | 2 lines (%K + %D) |\n| **Scale** | 0 to -100 | 0 to 100 |\n| **Speed** | Faster, more sensitive | Slower, more smooth |\n| **Best For** | Early signals | Confirmed signals |\n\n### Key Insight\n\nUse Williams %R to **spot** potential reversals early, then wait for Stochastic to **confirm** them!\n\n---

In [ ]:
# Calculate Williams %R\nif 'data' in locals() and not data.empty:\n    period = 14\n    \n    # Calculate Williams %R\n    highest_high = data['High'].rolling(window=period).max()\n    lowest_low = data['Low'].rolling(window=period).min()\n    \n    data['Williams_R'] = ((highest_high - data['Close']) / (highest_high - lowest_low)) * -100\n    \n    print("Williams %R calculated successfully!\\n")\n    print("Last 5 days:")\n    print(data[['Close', 'Williams_R']].tail())\n    \n    # Current reading\n    latest_wr = data['Williams_R'].iloc[-1]\n    \n    print(f"\\nCurrent Williams %R: {latest_wr:.2f}")\n    \n    if latest_wr > -20:\n        print("OVERBOUGHT - Above -20")\n    elif latest_wr < -80:\n        print("OVERSOLD - Below -80")\n    else:\n        print("NEUTRAL - Between -20 and -80")\nelse:\n    print("Please run data fetching cell first!")

In [ ]:
# Create comprehensive oscillator chart\nif 'data' in locals() and not data.empty and 'Stoch_K' in data.columns:\n    recent_data = data.tail(120)\n    \n    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(15, 12), sharex=True)\n    \n    # Price chart\n    ax1.plot(recent_data.index, recent_data['Close'], label='Close Price', linewidth=2, color='black')\n    ax1.set_ylabel('Price (RM)', fontsize=12)\n    ax1.set_title(f'{ticker} - Price with Oscillators', fontsize=16, fontweight='bold')\n    ax1.legend(loc='best')\n    ax1.grid(True, alpha=0.3)\n    \n    # Stochastic\n    ax2.plot(recent_data.index, recent_data['Stoch_K'], label='%K', linewidth=2, color='blue')\n    ax2.plot(recent_data.index, recent_data['Stoch_D'], label='%D', linewidth=2, color='red')\n    ax2.axhline(y=80, color='r', linestyle='--', alpha=0.5, label='Overbought (80)')\n    ax2.axhline(y=20, color='g', linestyle='--', alpha=0.5, label='Oversold (20)')\n    ax2.fill_between(recent_data.index, 80, 100, alpha=0.1, color='red')\n    ax2.fill_between(recent_data.index, 0, 20, alpha=0.1, color='green')\n    ax2.set_ylabel('Stochastic', fontsize=12)\n    ax2.set_ylim(0, 100)\n    ax2.legend(loc='best')\n    ax2.grid(True, alpha=0.3)\n    \n    # Williams %R\n    ax3.plot(recent_data.index, recent_data['Williams_R'], label='Williams %R', linewidth=2, color='purple')\n    ax3.axhline(y=-20, color='r', linestyle='--', alpha=0.5, label='Overbought (-20)')\n    ax3.axhline(y=-80, color='g', linestyle='--', alpha=0.5, label='Oversold (-80)')\n    ax3.fill_between(recent_data.index, -20, 0, alpha=0.1, color='red')\n    ax3.fill_between(recent_data.index, -100, -80, alpha=0.1, color='green')\n    ax3.set_ylabel('Williams %R', fontsize=12)\n    ax3.set_xlabel('Date', fontsize=12)\n    ax3.set_ylim(-100, 0)\n    ax3.legend(loc='best')\n    ax3.grid(True, alpha=0.3)\n    \n    plt.tight_layout()\n    plt.show()\n    \n    print("\\nChart shows price with both oscillators for comparison")\nelse:\n    print("Please run calculation cells first!")

---\n\n## Part 5: Trading Strategies with Oscillators\n\n### Strategy 1: Double Confirmation\n\n**Rules**:\n1. Wait for BOTH oscillators to agree\n2. Stochastic < 20 AND Williams %R < -80 → BUY signal\n3. Stochastic > 80 AND Williams %R > -20 → SELL signal\n4. Exit when either oscillator reaches opposite extreme\n\n**Strength**: Very reliable, few false signals\n**Weakness**: Slower, may miss some moves\n\n### Strategy 2: Stochastic Crossover\n\n**Rules**:\n1. Wait for %K to cross %D\n2. %K crosses ABOVE %D while both < 20 → BUY\n3. %K crosses BELOW %D while both > 80 → SELL\n4. Place stop-loss 2-3% away\n\n**Strength**: Clear entry signals\n**Weakness**: Can give false signals in strong trends\n\n### Strategy 3: Williams %R Early Entry\n\n**Rules**:\n1. Use Williams %R for early entry\n2. Williams %R < -80 → Prepare to buy\n3. Wait for Williams %R to cross ABOVE -80 → BUY\n4. Exit when Williams %R crosses ABOVE -20\n\n**Strength**: Catches moves early\n**Weakness**: More false signals, needs tight stops\n\n---

In [ ]:
# Implement double confirmation strategy\nif 'data' in locals() and not data.empty and 'Stoch_K' in data.columns:\n    # Detect signals\n    data['Buy_Signal'] = (data['Stoch_K'] < 20) & (data['Williams_R'] < -80)\n    data['Sell_Signal'] = (data['Stoch_K'] > 80) & (data['Williams_R'] > -20)\n    \n    # Find recent signals\n    recent_buys = data[data['Buy_Signal']].tail(5)\n    recent_sells = data[data['Sell_Signal']].tail(5)\n    \n    print("Double Confirmation Strategy Signals\\n")\n    print("="*60)\n    \n    print("\\nRecent BUY Signals (Both oversold):")\n    if len(recent_buys) > 0:\n        print(recent_buys[['Close', 'Stoch_K', 'Williams_R']].to_string())\n    else:\n        print("No recent buy signals")\n    \n    print("\\nRecent SELL Signals (Both overbought):")\n    if len(recent_sells) > 0:\n        print(recent_sells[['Close', 'Stoch_K', 'Williams_R']].to_string())\n    else:\n        print("No recent sell signals")\n    \n    # Current status\n    print("\\n" + "="*60)\n    print("CURRENT SETUP:")\n    \n    if data['Buy_Signal'].iloc[-1]:\n        print("BUY SIGNAL - Both oscillators oversold!")\n    elif data['Sell_Signal'].iloc[-1]:\n        print("SELL SIGNAL - Both oscillators overbought!")\n    else:\n        print("NO SIGNAL - Wait for both oscillators to agree")\nelse:\n    print("Please run calculation cells first!")

---

## Exercise 4: Calculate Stochastic Manually for a Specific Date

**Objective**: Reinforce your understanding of how the Stochastic formula works by calculating it by hand!

**Background**: You've seen the formula and code, but nothing beats calculating it yourself. This exercise will help you truly understand what the Stochastic oscillator is measuring.

**Your Task**:
1. Pick any recent date from our UUE Holdings data
2. Manually calculate %K for that date using the formula
3. Compare your result to the built-in calculation
4. Verify they match (within rounding)

**Starter Code**: Use the code cell below to complete this exercise.

**Expected Output**: Your manual calculation should match (or be very close to) the built-in calculation!

**Hint**: Remember that Stochastic compares where the current close is within the 14-day range. If close = lowest low, then %K = 0. If close = highest high, then %K = 100!

In [ ]:
# Exercise 4: Manual Stochastic Calculation
if "data" in locals() and not data.empty:
    # Step 1: Choose a date (you can change this!)
    chosen_date = data.index[-10]  # 10 days ago

    # Step 2: Get the closing price for that date
    # TODO: Get Close price for chosen_date
    current_close = None  # Replace with your code

    # Step 3: Find the lowest low in past 14 days (including chosen date)
    # Hint: Use data.loc[start_date:chosen_date, 'Low'].min()
    period = 14
    # TODO: Calculate start_date (14 days before chosen_date)
    start_date = None  # Replace with your code
    # TODO: Find lowest_low in that range
    lowest_low = None  # Replace with your code

    # Step 4: Find the highest high in past 14 days
    # TODO: Find highest_high in the same range
    highest_high = None  # Replace with your code

    # Step 5: Calculate %K using the formula
    # %K = (Current Close - Lowest Low) / (Highest High - Lowest Low) × 100
    # TODO: Calculate using formula
    manual_k = None  # Replace with your code

    # Step 6: Get the built-in calculation for comparison
    builtin_k = data.loc[chosen_date, "Stoch_K"]

    # Print results (uncomment after completing TODOs)
    # print(f"Date: {chosen_date.date()}")
    # print(f"Close: RM {current_close:.2f}")
    # print(f"Lowest Low (14d): RM {lowest_low:.2f}")
    # print(f"Highest High (14d): RM {highest_high:.2f}")
    # print(f"\nYour %K calculation: {manual_k:.2f}")
    # print(f"Built-in %K: {builtin_k:.2f}")
    # print(f"Difference: {abs(manual_k - builtin_k):.4f}")
else:
    print("Please run data fetching cells first!")

In [ ]:
# Validation Cell for Exercise 4
# Run this after completing Exercise 4 to check your answer

if "data" in locals() and "manual_k" in locals() and "builtin_k" in locals():
    # Check if calculation is correct (within 0.5% tolerance)
    difference = abs(manual_k - builtin_k)

    if difference < 0.5:
        print("✅ Great job! Your manual calculation matches the built-in Stochastic!")
        print(f"   Your understanding of the formula is solid!")
        print(f"   Difference: {difference:.4f} (excellent!)")
    elif difference < 2.0:
        print("✅ Good attempt! You're very close!")
        print(f"   Difference: {difference:.4f}")
        print("   Tip: Double-check your date range calculation")
    else:
        print("❌ Not quite right. Let's troubleshoot:")
        print(f"   Difference: {difference:.4f}")
        print("   Common mistakes:")
        print("   - Did you use the correct 14-day period?")
        print("   - Did you get the lowest LOW (not close)?")
        print("   - Did you get the highest HIGH (not close)?")
        print("   - Did you multiply by 100?")
else:
    print("⚠️  Please complete Exercise 4 first!")
    print("   Make sure to run the exercise code cell above.")

---

## Exercise 5: Identify and Visualize Divergences

**Objective**: Learn to spot divergences - powerful signals that often predict trend reversals!

**Background**: A **divergence** occurs when price and oscillator move in opposite directions:
- **Bullish Divergence**: Price makes new LOW, but Stochastic makes HIGHER low → Upward reversal coming
- **Bearish Divergence**: Price makes new HIGH, but Stochastic makes LOWER high → Downward reversal coming

**Why Divergences Matter**: They show weakening momentum BEFORE the price reverses - giving you advance warning!

**Your Task**:
1. Find a period where UUE Holdings made a significant low
2. Check if Stochastic showed a bullish divergence (higher low)
3. Visualize the divergence on a chart
4. Analyze what happened after the divergence

**Starter Code**: Use the code cell below to complete this exercise.

**Expected Output**:
- A chart showing price lows connected with Stochastic lows
- Clear identification of whether divergence exists
- Analysis of what happened after

**Hint**: Look for periods where the stock was falling (making lower lows in price), but Stochastic was rising (making higher lows). This is your divergence signal!

In [ ]:
# Exercise 5: Divergence Detection and Visualization
if "data" in locals() and not data.empty and "Stoch_K" in data.columns:
    # Step 1: Get recent 60 days for analysis
    recent = data.tail(60).copy()

    # Step 2: Find the two lowest price points
    # TODO: Use recent['Close'].nsmallest(2) to find 2 lowest prices
    two_lows_prices = None  # Replace with your code
    # TODO: Get their dates (index values)
    low_dates = None  # Replace with your code (list of 2 dates)

    # Step 3: Get Stochastic values at those same dates
    # TODO: Get Stoch_K values for the two dates
    stoch_at_lows = None  # Replace with your code (list of 2 values)

    # Step 4: Check for bullish divergence
    # If 2nd low in PRICE is lower BUT 2nd low in STOCH is higher = Bullish divergence!
    # TODO: Write condition to check this
    has_divergence = False  # Replace with your condition

    # Step 5: Create visualization
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10), sharex=True)

    # Plot price
    ax1.plot(recent.index, recent["Close"], linewidth=2, color="black", label="Close Price")
    # TODO: Mark the two low points on the price chart with red dots
    # Hint: Use ax1.scatter(low_dates, [prices], color='red', s=100, zorder=5)
    ax1.set_ylabel("Price (RM)", fontsize=12)
    ax1.set_title("Divergence Analysis - Price vs Stochastic", fontsize=14, fontweight="bold")
    ax1.legend()
    ax1.grid(True, alpha=0.3)

    # Plot Stochastic
    ax2.plot(recent.index, recent["Stoch_K"], linewidth=2, color="blue", label="Stochastic %K")
    ax2.axhline(y=20, color="g", linestyle="--", alpha=0.5)
    # TODO: Mark the two low points on the Stochastic chart with green dots
    ax2.set_ylabel("Stochastic %K", fontsize=12)
    ax2.set_xlabel("Date", fontsize=12)
    ax2.legend()
    ax2.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    # Print analysis
    print("\nDivergence Analysis:")
    print("=" * 60)
    # TODO: Print the dates and values
    # TODO: Print whether bullish divergence was detected
    # TODO: Print what happened to price after the divergence

else:
    print("Please run all previous cells first!")

In [ ]:
# Validation Cell for Exercise 5
# Run this after completing Exercise 5 to check your answer

if "data" in locals() and "recent" in locals():
    # Check if student identified the divergence correctly
    print("✅ Exercise 5 Validation")
    print("=" * 60)

    # Get the actual lowest points for reference
    recent_check = data.tail(60).copy()
    two_lows = recent_check["Close"].nsmallest(2)

    if len(two_lows) >= 2:
        dates_check = two_lows.index
        prices_check = two_lows.values
        stoch_values = recent_check.loc[dates_check, "Stoch_K"].values

        # Check for divergence
        price_trend = "falling" if prices_check[1] < prices_check[0] else "rising"
        stoch_trend = "rising" if stoch_values[1] > stoch_values[0] else "falling"

        has_bullish_divergence = price_trend == "falling" and stoch_trend == "rising"

        print("\nReference Answer:")
        print(
            f"First Low:  {dates_check[0].date()} - Price: RM {prices_check[0]:.2f}, Stoch: {stoch_values[0]:.2f}"
        )
        print(
            f"Second Low: {dates_check[1].date()} - Price: RM {prices_check[1]:.2f}, Stoch: {stoch_values[1]:.2f}"
        )
        print(f"\nPrice trend: {price_trend}")
        print(f"Stochastic trend: {stoch_trend}")

        if has_bullish_divergence:
            print("\n✅ BULLISH DIVERGENCE DETECTED!")
            print("   This is a potential buy signal - price falling but momentum strengthening!")
        else:
            print("\n❌ No bullish divergence in this period")
            print("   Both price and Stochastic are moving in the same direction")

        print("\n💡 Key Learning:")
        print("   Divergences are advance warnings of trend reversals.")
        print("   Always wait for price confirmation before trading!")
    else:
        print("Not enough data points to validate")
else:
    print("⚠️  Please complete Exercise 5 first!")
    print("   Make sure to run the exercise code cell above.")

---\n\n## Practice Exercises\n\n### Exercise 1: Oscillator Comparison\nLook at the charts and answer:\n1. When did Stochastic give false signals?\n2. Did Williams %R signal earlier than Stochastic?\n3. Which oscillator do you find easier to interpret?\n\n### Exercise 2: Try Different Stocks\nAnalyze different stocks and compare oscillator behavior:\n- **1155.KL** (Maybank - large cap)\n- **5296.KL** (Tenaga - utility)\n\nQuestions:\n1. Which stock shows more overbought/oversold signals?\n2. Do oscillators work better on volatile or stable stocks?\n\n### Exercise 3: Backtest a Strategy\nChoose the Double Confirmation strategy:\n1. Find last 3 buy signals\n2. Check what happened to price after each signal\n3. Calculate win rate (how many were profitable?)\n\n---\n\n## What's Next?\n\nCongratulations on completing Module 08!\n\n**Next Module: Fibonacci Retracements & Support/Resistance Zones**\n\nIn Module 09, you'll learn:\n- Fibonacci retracement levels\n- Support and resistance zones\n- Combining Fibonacci with oscillators\n- Entry and exit optimization\n\n**File**: `09_fibonacci_support_resistance.ipynb`\n\n---\n\n**Happy Trading! Always practice with paper money first!**